## Modeling a Machine Learning App for Poultry Diseases Diagnostics
* The dataset of poultry [fecal images](http://doi.org/10.5281/zenodo.4628934) is used for training

* Image classification task with [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/tutorials/model_maker_image_classification)
* Model Architecture: MobileNet
* The end-to-end model will be deployed on Android mobile device for detection of Salmonella and Coccidiosis diseases and the Healthy poultry


## Prerequisites

We first need to install several required packages, including Model Maker package that in GitHub [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!pip install tflite-model-maker

Import the required packages.

In [2]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

### Load Dataset
* Get the data path for the poultry diseases dataset
* We will use 500 poultry fecal images for each class 
* The model is trained for three classes of:
   - Salmonela disease (`salmo`)
   - Coccidiosis disease (`cocci`) and
   - Normal (`healthy`)


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
image_path = '/content/gdrive/MyDrive/poultryFecal_images'

In [5]:
os.listdir(image_path)

['healthy', 'salmo', 'cocci']

### Training on MOBILENET V1 


### Step 1: Load Input Data Specific to an On-device ML App

* The poultry dataset contains 1500 fecal images belonging to 3 classes. 
* Train and test split is 0.9/0.1

* The dataset has the following directory structure:

<pre>
<b>poultryFecal_images</b>
|__ <b>cocci</b>
    |______ cocci.0.jpg
    |______ cocci.1.jpg
    |______ ...
|__ <b>healthy</b>
    |______ healthy.0.jpg
    |______ healthy.1.jpg
    |______ ...
|__ <b>salmo</b>
    |______ salmo.0.jpg
    |______ salmo.1.jpg
    |______ ...
</pre>

In [9]:
data = ImageClassifierDataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 1500, num_label: 3, labels: cocci, healthy, salmo.


### Step 2. Customize and Train the TensorFlow model.

In [10]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 3,416,867
Trainable params: 3,843
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
42/42 [==============================] - 378s 9s/step - loss: 0.8247 - accuracy: 0.6755
Epoch 2/5
42/42 [==============================] - 86s 2s/step - loss: 0.4455 - accuracy: 0.9361
Epoch 3/5
42/42 [==============================] - 83s 2s/step - loss: 0.4120 - accuracy: 0.9573
Epoch 4/5
42/42 [=================

### Step 3. Evaluate the model.

In [11]:
loss, accuracy = model.evaluate(test_data)

5/5 [==============================] - 110s 6s/step - loss: 0.3885 - accuracy: 0.9667


### Step 4.  Export to TensorFlow Lite model.

Here, we export TensorFlow Lite model with [metadata](https://www.tensorflow.org/lite/convert/metadata) which provides a standard for model descriptions. The label file is embedded in metadata.

You could download it in the left sidebar same as the uploading part for your own use.

In [12]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: /tmp/tmpmlkz2k66/assets


INFO:tensorflow:Assets written to: /tmp/tmpmlkz2k66/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp5ajstu27/labels.txt.


INFO:tensorflow:Saving labels in /tmp/tmp5ajstu27/labels.txt.


### Training on MOBILENET V2

For comparison with MobileNet V1 above

In [13]:
model = image_classifier.create(train_data, model_spec=model_spec.mobilenet_v2_spec, epochs=5)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_1 (HubK (None, 1280)              2257984   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/5
42/42 [==============================] - 87s 2s/step - loss: 0.9542 - accuracy: 0.6528
Epoch 2/5
42/42 [==============================] - 82s 2s/step - loss: 0.4441 - accuracy: 0.9638
Epoch 3/5
42/42 [==============================] - 83s 2s/step - loss: 0.4211 - accuracy: 0.9559
Epoch 4/5
42/42 [==============================] - 84s 2s/step - loss: 0.4

In [14]:
loss, accuracy = model.evaluate(test_data)

5/5 [==============================] - 25s 2s/step - loss: 0.3883 - accuracy: 0.9667


### Step 5. Deployment.
* MobileNet V2 model has higher accuracy on training but same accuracy on testing compared to MobileNet V1.

* Deployment used MobileNet V1 model

* The TensorFlow Lite model file (`model.tflite`) was deployed on Android using the in on-device application [image classification](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification) reference app.